In [ ]:
# Notebook 2: Embedding and FAISS Index
# Ce notebook montre comment créer des embeddings et un index FAISS

# Cellule 1: Imports
import sys
sys.path.append('../src')

from modules.embeddings import EmbeddingModel
from modules.retrieval import FAISSRetriever
from modules.config import config
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

print("✅ Modules importés")

# Cellule 2: Initialiser le modèle d'embedding
embedding_model = EmbeddingModel()
print(f"Modèle: {embedding_model.model_name}")
print(f"Dimension: {embedding_model.get_embedding_dimension()}")

# Cellule 3: Tester avec des exemples simples
sample_texts = [
    "L'intelligence artificielle transforme notre monde",
    "Le machine learning permet aux machines d'apprendre",
    "Les réseaux de neurones sont inspirés du cerveau humain",
    "Python est un langage de programmation populaire",
    "La cuisine française est réputée dans le monde entier"
]

# Générer les embeddings
embeddings = embedding_model.encode(sample_texts)
print(f"\n📊 Shape des embeddings: {embeddings.shape}")
print(f"Type: {embeddings.dtype}")

# Cellule 4: Visualiser les embeddings (réduction en 2D avec PCA)
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings)

plt.figure(figsize=(10, 8))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], s=100)

for i, txt in enumerate(sample_texts):
    plt.annotate(
        txt[:30] + "...", 
        (embeddings_2d[i, 0], embeddings_2d[i, 1]),
        fontsize=9
    )

plt.title("Visualisation des embeddings (PCA 2D)")
plt.xlabel("Composante 1")
plt.ylabel("Composante 2")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Cellule 5: Calculer les similarités
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(embeddings)
print("\n🔍 Matrice de similarité cosinus:")
print(similarities)

# Trouver les paires les plus similaires
for i in range(len(sample_texts)):
    for j in range(i+1, len(sample_texts)):
        sim = similarities[i][j]
        print(f"\nSimilarité entre:")
        print(f"  '{sample_texts[i][:40]}'")
        print(f"  '{sample_texts[j][:40]}'")
        print(f"  Score: {sim:.4f}")

# Cellule 6: Créer un index FAISS
retriever = FAISSRetriever()

# Créer l'index avec nos exemples
metadata = [
    {'chunk_id': f'chunk_{i}', 'content': txt, 'document_name': 'example.txt'}
    for i, txt in enumerate(sample_texts)
]

retriever.create_index(embeddings, metadata)
print(f"\n✅ Index FAISS créé avec {retriever.index.ntotal} vecteurs")

# Cellule 7: Tester la recherche
query = "Comment fonctionnent les réseaux de neurones ?"
print(f"\n🔍 Question: {query}")

results = retriever.search(query, top_k=3)

print("\n📋 Résultats:")
for i, result in enumerate(results, 1):
    print(f"\n{i}. Score: {result['score']:.4f}")
    print(f"   Contenu: {result['content']}")
    print(f"   Chunk ID: {result['chunk_id']}")

# Cellule 8: Charger des chunks réels (du notebook précédent)
import json

chunks_file = config.CHUNKS_DIR / "votre_document.pdf_chunks.json"

if chunks_file.exists():
    with open(chunks_file, 'r', encoding='utf-8') as f:
        chunks = json.load(f)
    
    print(f"\n📄 Chunks chargés: {len(chunks)}")
    
    # Générer les embeddings
    chunk_texts = [chunk['content'] for chunk in chunks]
    chunk_embeddings = embedding_model.encode(chunk_texts)
    
    # Créer l'index
    retriever_full = FAISSRetriever()
    retriever_full.create_index(chunk_embeddings, chunks)
    
    print(f"✅ Index créé avec {retriever_full.index.ntotal} vecteurs")
    
    # Sauvegarder
    retriever_full.save_index()
    print(f"✅ Index sauvegardé dans {config.INDEX_DIR}")
else:
    print("⚠️ Aucun fichier de chunks trouvé. Exécutez d'abord le notebook 01.")

# Cellule 9: Test de recherche sur l'index complet
if 'retriever_full' in locals():
    test_query = "Quel est le sujet principal du document ?"
    print(f"\n🔍 Question: {test_query}")
    
    results = retriever_full.search(test_query, top_k=5)
    
    print("\n📋 Top 5 résultats:")
    for i, result in enumerate(results, 1):
        print(f"\n{i}. {result['document_name']} - Chunk {result['chunk_index']}")
        print(f"   Score: {result['score']:.4f}")
        print(f"   Extrait: {result['content'][:150]}...")

# Cellule 10: Statistiques de l'index
if 'retriever_full' in locals():
    print("\n📊 Statistiques de l'index:")
    print(f"Nombre total de vecteurs: {retriever_full.index.ntotal}")
    print(f"Dimension des vecteurs: {retriever_full.dimension}")
    print(f"Nombre de documents uniques: {len(set(c['document_name'] for c in retriever_full.metadata))}")
    
    # Distribution des chunks par document
    from collections import Counter
    doc_counts = Counter(c['document_name'] for c in retriever_full.metadata)
    
    print("\n📚 Répartition des chunks par document:")
    for doc, count in doc_counts.items():
        print(f"  {doc}: {count} chunks")